In [9]:
import os
from urllib.parse import urljoin
import pystac
import datetime

from pystac import (
    Summaries,
)
from pystac.extensions.datacube import (
    DatacubeExtension,
    DimensionType,
    HorizontalSpatialDimensionAxis,
)

from pystac.extensions.projection import ProjectionExtension
from pystac.utils import str_to_datetime
from shapely.geometry import mapping, box


import requests
from requests.auth import HTTPBasicAuth

from dotenv import load_dotenv
load_dotenv("/home/otto/s1_zarr/.env")

True

In [10]:
username = os.getenv("username_stac_prod")
password = os.getenv("password_stac_prod")

In [11]:
def create_collection():

    """Creation of STAC collection"""

    datacube = {
    "x": {
        "axis": HorizontalSpatialDimensionAxis.X,
        "type": DimensionType.SPATIAL,
        "extent": [112500, 695500],
    },
    "y": {
        "axis": HorizontalSpatialDimensionAxis.Y,
        "type": DimensionType.SPATIAL,
        "extent": [258500, 586500],
    },
    "time": {
    "type": DimensionType.TEMPORAL,
    "extent": ["1961-01-01T00:00:00Z", None],
},
}

    parameters = {
        "TN": {
            "description": "minimum temperature",
            "unit": "degree Celsius",
            "dimensions": ["x", "y", "time"],
        },
        "TX": {
            "description": "maximum temperature",
            "unit": "degree Celsius",
            "dimensions": ["x", "y", "time"],
        },
        "RR": {
            "description": "relative humidity",
            "unit": "p",
            "dimensions": ["x", "y", "time"],
        },
        "RR": {
            "description": "precipitation sum",
            "unit": "kg m-2",
            "dimensions": ["x", "y", "time"],
        },
        "SA": {
            "description": "duration of sunshine",
            "unit": "s",
            "dimensions": ["x", "y", "time"],
        },
    }

    collection = pystac.Collection(
        id="spartacus-daily",
        title= "Spartacus Analysis Daily (1 km)",
        description="This gridded dataset, stored in Zarr format, contains daily observations of air temperature (minimum: TN, maximum: TX), precipitation (RR), and sunshine duration (SA) over Austria at 1 km resolution, starting in 1961. It also includes monthly, seasonal, and annual aggregates. The dataset is operationally updated on a daily basis.",
        extent=pystac.Extent(
            pystac.SpatialExtent([[9.393909, 46.161327, 17.38218, 49.17754]]),
            pystac.TemporalExtent([[str_to_datetime("1961-01-01T00:00:00Z"), None]]),
        ),
        keywords=["GeoSphere", "Spartacus", "sunshine", "pressure", "precipitation", "temperature"],
        license="CC-BY-4.0",
        extra_fields={"cube:dimensions": datacube,
                      "cube:variables": parameters},
    )

    collection.providers = [
    pystac.Provider(
        name="EODC",
        roles=[
            pystac.ProviderRole.HOST,
        ],
        url="https://eodc.eu/",
        ),
        pystac.Provider(
        name="GeoSphere Austria",
        roles=[
            pystac.ProviderRole.PRODUCER,
            pystac.ProviderRole.LICENSOR,
            pystac.ProviderRole.PROCESSOR,
        ],
        url="https://data.hub.geosphere.at/",
    ),
]

    collection.stac_extensions = [
        DatacubeExtension.get_schema_uri(),
        ProjectionExtension.get_schema_uri(),
    ]

    summaries_dict = {}
    summaries_dict["Doi"] = ["https://doi.org/10.60669/m6w8-s545"]
    summaries_dict["Epsg"] = ["3416"]
    summaries_dict["Projection"] = ["Austria Lambert (EPSG: 3416)"]
    summaries_dict["Timezone"] = ["UTC"]
    summaries_dict["Grid"] = ["1x1km"]
    summaries_dict["Temporal Resolution"] = ["daily"]
    collection.summaries = Summaries(summaries_dict)

    collection.add_asset(
    key="thumbnail",
    asset=pystac.Asset(
        href="https://raw.githubusercontent.com/katharinastarzer21/repository_bsp/main/january_2024_map.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Thumbnail"
    )
)

    bbox = [9.393909, 46.16133, 17.38218, 49.17754]
    properties = {"gsd": 1000,
                  "chunksize": {"x":584, "y":329, "time": 365},
                  "proj:bbox": [112500, 258500, 695500, 586500],
                  "proj:wkt2": "PROJCS[\"ETRS89 / Austria Lambert\",GEOGCS[\"ETRS89\",DATUM[\"European_Terrestrial_Reference_System_1989\",SPHEROID[\"GRS 1980\",6378137,298.257222101,AUTHORITY[\"EPSG\",\"7019\"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY[\"EPSG\",\"6258\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4258\"]],PROJECTION[\"Lambert_Conformal_Conic_2SP\"],PARAMETER[\"standard_parallel_1\",49],PARAMETER[\"standard_parallel_2\",46],PARAMETER[\"latitude_of_origin\",47.5],PARAMETER[\"central_meridian\",13.33333333333333],PARAMETER[\"false_easting\",400000],PARAMETER[\"false_northing\",400000],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AUTHORITY[\"EPSG\",\"3416\"]]",
                  "proj:shape": [584, 329],
                  "proj:geometry": mapping(box(*bbox)),
                  "proj:transform": [1000.0, 0.0, 112500.0, 0.0, 1000.0, 258500.0, 0.0, 0.0, 1.0],
                  "cube:dimensions": datacube,}
    
    collection.extra_fields.update(properties)
    
    item = pystac.Item(id="zarr_store", geometry=mapping(box(*bbox)), bbox=bbox, datetime=datetime.datetime(1961,1,1), 
                       properties=properties)
    collection.add_asset("zarr_store",
                   pystac.Asset(href="https://data.eodc.eu/collections/SPARTACUS/SPARTACUS.zarr",
                                media_type=pystac.MediaType.ZARR,
                                title="SPARTACUS",
                                roles=["data"]),)

    destination = "inca_stac_coll"
    collection_path = os.path.join(destination, "{}.json".format(collection.id))
    collection.set_self_href(collection_path)
    item.save_object(dest_href=os.path.join(destination, "spartacus_item.json"))
    collection.save_object()
    
    return collection, item

## Create Collection
create_collection()
k, item = create_collection()
k.to_dict()

app_host = "https://stac.eodc.eu/ingestion/v1"
url = urljoin(app_host, "/collections")



r = requests.post(app_host + "/collections", json=k.to_dict(), auth=HTTPBasicAuth(username, password), timeout = 30)
#r = requests.post("https://dev.stac.eodc.eu/ingestion/v1/collections/spartacus-daily/items", json=item.to_dict(), auth=HTTPBasicAuth(username, password), timeout = 30)


In [ ]:
#r = requests.delete('https://dev.stac.eodc.eu/ingestion/v1/collections/spartacus-daily',auth=HTTPBasicAuth(username, password), timeout = 30)


In [11]:
collection = pystac.Collection.from_file("inca_stac_coll/spartacus-daily.json")

try:
    collection.validate()
    print("Collection is valid.")
except Exception as e:
    print("Validation failed:", e)


Collection is valid.


In [ ]:
item = pystac.Item.from_file("inca_stac_coll/spartacus_item.json")

try:
    item.validate()
    print("Item is valid.")
except Exception as e:
    print("Validation failed:", e)


Item is valid.
